##  Problem Statement

In this programming assignment, your task is to classify geographical locations according to their predicted tree cover using Gradient Boosting and Random Forest classifiers. You are expected to fill in functions that would complete this task. All of the necessary helper code is included in this notebook. However, we advise you to go over the slides, lecture material, the EdX videos and the corresponding notebooks before you attempt this Programming Assignment. You can find information about the dataset to be used in the following links:

* **Dataset:** http://archive.ics.uci.edu/ml/datasets/Covertype 

* **Dataset description:** http://archive.ics.uci.edu/ml/machine-learning-databases/covtype/covtype.info

##  Notebook Setup

In [1]:
# To time the entire solution
import time
start_nb = time.time()

In [2]:
import os

#os.environ["PYSPARK_PYTHON"]="python3"
#os.environ["PYSPARK_DRIVER_PYTHON"] = "python3"
import findspark
findspark.init()
from pyspark import SparkContext
sc=SparkContext()

In [3]:
from pyspark.mllib.linalg import Vectors
from pyspark.mllib.regression import LabeledPoint

from pyspark.mllib.tree import GradientBoostedTrees, GradientBoostedTreesModel
from pyspark.mllib.tree import RandomForest, RandomForestModel
from pyspark.mllib.util import MLUtils

# import os
import pickle
from os.path import exists

%config IPCompleter.greedy=True

In [4]:
#define a dictionary of cover types
CoverTypes={1.0: 'Spruce/Fir',
            2.0: 'Lodgepole Pine',
            3.0: 'Ponderosa Pine',
            4.0: 'Cottonwood/Willow',
            5.0: 'Aspen',
            6.0: 'Douglas-fir',
            7.0: 'Krummholz' }
print('Tree Cover Types:', CoverTypes)

Tree Cover Types: {1.0: 'Spruce/Fir', 2.0: 'Lodgepole Pine', 3.0: 'Ponderosa Pine', 4.0: 'Cottonwood/Willow', 5.0: 'Aspen', 6.0: 'Douglas-fir', 7.0: 'Krummholz'}


## Collecting Data

In [5]:
# Break up features that are made out of several binary features.
def get_columns(cols_txt):
    cols=[a.strip() for a in cols_txt.split(',')]
    colDict={a:[a] for a in cols}
    colDict['Soil_Type (40 binary columns)'] = ['ST_'+str(i) for i in range(40)]
    colDict['Wilderness_Area (4 binarycolumns)'] = ['WA_'+str(i) for i in range(4)]
    columns=[]
    for item in cols:
        columns = columns + colDict[item]
    return columns
    #print(columns)

In [6]:
# Define the feature names
cols_txt="""
Elevation, Aspect, Slope, Horizontal_Distance_To_Hydrology,
Vertical_Distance_To_Hydrology, Horizontal_Distance_To_Roadways,
Hillshade_9am, Hillshade_Noon, Hillshade_3pm,
Horizontal_Distance_To_Fire_Points, Wilderness_Area (4 binarycolumns), 
Soil_Type (40 binary columns), Cover_Type
"""
columns = get_columns(cols_txt)

In [7]:
# Read the file into an RDD
# When using sc.textRead you need to use an absolute path.
# If doing this on a real cluster, you need the file to be available on all nodes, ideally in HDFS.
path='covtype/covtype.data'
inputRDD=sc.textFile(path)

## Helper Functions
Here are some helper functions that you will have to fill up.

### label_RDD

The function <font color="blue">label_RDD</font> takes an RDD as input and returns an RDD of labeled points

Input: RDD consisting of a string with comma separated values (InputRDD)


**<font color="magenta" size=2>Example Input</font>**
``` python
'2596,51,3,258,0,510,221,232,148,6279,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,5'
```
Output: RDD of the type LabeledPoint with the first element being the label and second element being a DenseVector that contains all the elements of the InputRDD(Except the last value which is the label).

**<font color="blue" size=2>Example Output</font>**
``` python
LabeledPoint(5.0, [2596.0,51.0,3.0,258.0,0.0,510.0,221.0,232.0,148.0,6279.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0])
```



In [8]:
## Insert your answer in this cell. DO NOT CHANGE THE NAME OF THE FUNCTION.
def label_RDD(inputRDD):
    ###
    ### YOUR CODE HERE
    ###
    Data = inputRDD.map(lambda s: [float(d) for d in s.split(',')]).map(lambda d: LabeledPoint(d[-1], Vectors.dense(d[:-1])))
    
    return Data

In [9]:
Data = label_RDD(inputRDD)
# Data.cache()

In [10]:
assert Data.first().label == 5.0
assert Data.first().features == Vectors.dense([2596.0, 51.0, 3.0, 258.0, 0.0, 510.0, 221.0, 232.0, 148.0, 6279.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0])

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.runJob.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 0.0 failed 1 times, most recent failure: Lost task 0.0 in stage 0.0 (TID 0, host.docker.internal, executor driver): java.net.SocketException: Connection reset by peer: socket write error
	at java.net.SocketOutputStream.socketWrite0(Native Method)
	at java.net.SocketOutputStream.socketWrite(Unknown Source)
	at java.net.SocketOutputStream.write(Unknown Source)
	at java.io.BufferedOutputStream.flushBuffer(Unknown Source)
	at java.io.BufferedOutputStream.write(Unknown Source)
	at java.io.DataOutputStream.write(Unknown Source)
	at java.io.FilterOutputStream.write(Unknown Source)
	at org.apache.spark.api.python.PythonRDD$.writeUTF(PythonRDD.scala:465)
	at org.apache.spark.api.python.PythonRDD$.write$1(PythonRDD.scala:285)
	at org.apache.spark.api.python.PythonRDD$.$anonfun$writeIteratorToStream$1(PythonRD

### count_examples

The function <font color="blue">count_examples</font> takes an RDD as input and returns count of number of labels belonging to each class

Input: RDD obtained as the output of the labelRDD


**<font color="magenta" size=2>Example Input</font>**
``` python
[LabeledPoint(5.0, [2596.0,51.0,3.0,258.0,0.0,510.0,221.0,232.0,148.0,6279.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0]),
 LabeledPoint(5.0, [2590.0,56.0,2.0,212.0,-6.0,390.0,220.0,235.0,151.0,6225.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0]),
 LabeledPoint(2.0, [2804.0,139.0,9.0,268.0,65.0,3180.0,234.0,238.0,135.0,6121.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0])]
```
Output: list of tuples (label, count)

**NOTE: The outputs need to be sorted in descending order by counts**

**<font color="blue" size=2>Example Output</font>**
``` python
[(5.0, 2), (2.0, 1)]
```

In [11]:
## Insert your answer in this cell. DO NOT CHANGE THE NAME OF THE FUNCTION.
def count_examples(Data):
    ###
    ### YOUR CODE HERE
    ###
    return sorted(Data.map(lambda p: p.label).countByValue().items(), key=lambda x:-x[1])

In [12]:
counts = count_examples(Data)

In [13]:
counts3 = count_examples(sc.parallelize(Data.take(3)))

In [14]:
assert type(counts3) == list, 'Incorrect return type'
assert type(counts3[0]) == tuple, 'Incorrect return type'
assert type(counts3[0][0]) == float, 'Incorrect return type'
assert type(counts3[0][1]) == int, 'Incorrect return type'

In [15]:
assert counts3[0][0] == 5.0, 'Incorrect return value'
assert counts3[0][1] == 2, 'Incorrect return value'

In [16]:
# Hidden Tests Here
###
### AUTOGRADER TEST - DO NOT REMOVE
###


In [17]:
total=Data.count()
print('total data size=',total)
print('              type (label):   percent of total')
print('---------------------------------------------------------')
print('\n'.join(['%20s (%3.1f):\t%4.2f'%(CoverTypes[a[0]],a[0],100.0*a[1]/float(total)) for a in counts]))

total data size= 581012
              type (label):   percent of total
---------------------------------------------------------
      Lodgepole Pine (2.0):	48.76
          Spruce/Fir (1.0):	36.46
      Ponderosa Pine (3.0):	6.15
           Krummholz (7.0):	3.53
         Douglas-fir (6.0):	2.99
               Aspen (5.0):	1.63
   Cottonwood/Willow (4.0):	0.47


### labels_to_binary (Making the problem binary)

The implementation of BoostedGradientTrees in MLLib supports only binary problems. the `CovType` problem has
7 classes. To make the problem binary we choose the `Lodgepole Pine` (label = 2.0). We therefore transform the dataset to a new dataset where the label is `1.0` is the class is `Lodgepole Pine` and is `0.0` otherwise.

The function <font color="blue">labels_to_binary</font> takes an RDD as input and returns an RDD with binary labels
such that: 

```python
if label == 2:      #Since label 2 has the highest count value
    new_label = 1
    
else:
    new_label = 0
```

Input: Labelled RDD (Output from <font color="blue">label_RDD</font> function)


**<font color="magenta" size=2>Example Input</font>**
``` python
LabeledPoint(5.0, [2596.0,51.0,3.0,258.0,0.0,510.0,221.0,232.0,148.0,6279.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0])
```
Output: The same RDD with label of all entries as 0 except for label = 2.0 where label becomes 1.0

**<font color="blue" size=2>Example Output</font>**
``` python
LabeledPoint(0.0, [2596.0,51.0,3.0,258.0,0.0,510.0,221.0,232.0,148.0,6279.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0])
```

In [18]:
## Insert your answer in this cell. DO NOT CHANGE THE NAME OF THE FUNCTION.
def labels_to_binary(Data):
    ###
    ### YOUR CODE HERE
    ###
    
    return Data.map(lambda p: LabeledPoint(1.0, p.features) if p.label == 2.0 else LabeledPoint(0.0, p.features))

In [19]:
Data = labels_to_binary(Data)

In [20]:
assert Data.first().label == 0.0

### Reducing data size
For this assignment, we will use only 10% of the original data.

In [21]:
trainingData = sc.parallelize(pickle.load(open('training10p.pkl', 'rb')))
testData = sc.parallelize(pickle.load(open('test10p.pkl', 'rb')))

In [22]:
print('Sizes: Data1=%d, trainingData=%d, testData=%d'%(trainingData.cache().count() + testData.cache().count(),trainingData.cache().count(),testData.cache().count()))

Sizes: Data1=58100, trainingData=40682, testData=17418


In [23]:
counts = count_examples(testData)

## Gradient Boosted Trees

* Following [this example](http://spark.apache.org/docs/latest/mllib-ensembles.html#classification) from the mllib documentation

* [pyspark.mllib.trees documentation](http://spark.apache.org/docs/latest/api/python/pyspark.mllib.html#pyspark.mllib.tree.GradientBoostedTrees)

### Main classes and methods

* `GradientBoostedTrees` is the class that implements the learning trainClassifier,
   * It's main method is `trainClassifier(trainingData)` which takes as input a training set and generates an instance of `GradientBoostedTreesModel`
   * The main parameter from train Classifier are:
      * **data** – Training dataset: RDD of LabeledPoint. Labels should take values {0, 1}.
      * categoricalFeaturesInfo – Map storing arity of categorical features. E.g., an entry (n -> k) indicates that feature n is categorical with k categories indexed from 0: {0, 1, ..., k-1}.
      * **loss** – Loss function used for minimization during gradient boosting. Supported: {“logLoss” (default), “leastSquaresError”, “leastAbsoluteError”}.
      * **numIterations** – Number of iterations of boosting. (default: 100 **PLEASE USE numIterations=10 FOR THIS ASSIGNMENT**)
      * **learningRate** – Learning rate for shrinking the contribution of each estimator. The learning rate should be between in the interval (0, 1]. (default: 0.1)
      * **maxDepth** – Maximum depth of the tree. E.g., depth 0 means 1 leaf node; depth 1 means 1 internal node + 2 leaf nodes. (default: 3)
      * **maxBins** – maximum number of bins used for splitting features (default: 32) DecisionTree requires maxBins >= max categories
      
      
* `GradientBoostedTreesModel` represents the output of the boosting process: a linear combination of classification trees. The methods supported by this class are:
   * `save(sc, path)` : save the tree to a given filename, sc is the Spark Context.
   * `load(sc,path)` : The counterpart to save - load classifier from file.
   * `predict(X)` : predict on a single datapoint (the `.features` field of a `LabeledPont`) or an RDD of datapoints.
   * `toDebugString()` : print the classifier in a human readable format.

### Example

The function <font color="blue">Classify_GB</font> takes as inputs:

1. **trainingData**: Training data (Type: RDD)
2. **testData**: Test data (Type: RDD)
3. **maxDepth**: Depth of tree (Type: int)

The function trains a GradientBoostedTrees classifier and returns the error

**Output**: error (Type: float)

**<font color="blue" size=2>Example Output</font>**
``` python
error=0.3
```

### Definition

In [28]:
## Insert your answer in this cell. DO NOT CHANGE THE NAME OF THE FUNCTION.
def Classify_GB(trainingData, testData, maxDepth):
    ###
    ### YOUR CODE HERE
    ###
    model = GradientBoostedTrees.trainClassifier(trainingData, {}, maxDepth=maxDepth)
    pred = model.predict(testData.map(lambda x: x.features))
    lp = testData.map(lambda lp: lp.label).zip(pred)
    return lp.map(lambda lp: (lp[0] - lp[1])*(lp[0] - lp[1])).sum()/float(testData.count())

### Test cases

In [29]:
visible_results=pickle.load(open('GradientBoostingResultsVisible.pkl','rb'))
assert Classify_GB(trainingData, testData, 1) <= visible_results['B_10p_1'] 

In [30]:
assert Classify_GB(trainingData, testData, 3) <= visible_results['B_10p_3']

In [31]:
#Hidden Tests here
###
### AUTOGRADER TEST - DO NOT REMOVE
###


In [32]:
#Hidden Tests here
###
### AUTOGRADER TEST - DO NOT REMOVE
###


## Random Forests

### Introduction

* Following [this example](http://spark.apache.org/docs/latest/mllib-ensembles.html#classification) from the mllib documentation

* [pyspark.mllib.trees.RandomForest documentation](http://spark.apache.org/docs/latest/api/python/pyspark.mllib.html#pyspark.mllib.tree.RandomForest)

**trainClassifier**`(data, numClasses, categoricalFeaturesInfo, numTrees, featureSubsetStrategy='auto', impurity='gini', maxDepth=4, maxBins=32, seed=None)`   
Method to train a decision tree model for binary or multiclass classification.

**Parameters:**  
* *data* – Training dataset: RDD of LabeledPoint. Labels should take values {0, 1, ..., numClasses-1}.  
* *numClasses* – number of classes for classification.  
* *categoricalFeaturesInfo* – Map storing arity of categorical features. E.g., an entry (n -> k) indicates that feature n is categorical with k categories indexed from 0: {0, 1, ..., k-1}.  
* *numTrees* – Number of trees in the random forest.  
* *featureSubsetStrategy* – Number of features to consider for splits at each node. Supported: “auto” (default), “all”, “sqrt”, “log2”, “onethird”. If “auto” is set, this parameter is set based on numTrees: if numTrees == 1, set to “all”; if numTrees > 1 (forest) set to “sqrt”.
* *impurity* – Criterion used for information gain calculation. Supported values: “gini” (recommended) or “entropy”.  
* *maxDepth* – Maximum depth of the tree. E.g., depth 0 means 1 leaf node; depth 1 means 1 internal node + 2 leaf nodes. (default: 4)  
* *maxBins* – maximum number of bins used for splitting features (default: 32)
* *seed* – Random seed for bootstrapping and choosing feature subsets.  

**Returns:**	
RandomForestModel that can be used for prediction

### Example
The function <font color="blue">Classify_RF</font> takes as inputs:

1. **trainingData**: Training data (Type: RDD)
2. **testData**: Test data (Type: RDD)
3. **maxDepth**: Depth of tree (Type: int)

The function trains a RandomForest classifier and returns the error in classification

**Output**: error (Type: float)

**<font color="blue" size=2>Example Output</font>**
``` python
error=0.3
```

Note: You are allowed to alter the number of trees parameter

### Definition

In [35]:
## Insert your answer in this cell. DO NOT CHANGE THE NAME OF THE FUNCTION.
def Classify_RF(trainingData, testData, depth):    
    ###
    ### YOUR CODE HERE
    ###
    model = RandomForest.trainClassifier(trainingData, 2, {}, 3, maxDepth=depth)
    pred = model.predict(testData.map(lambda x: x.features))
    lp = testData.map(lambda lp: lp.label).zip(pred)
    return lp.map(lambda lp: (lp[0] - lp[1])*(lp[0] - lp[1])).sum()/float(testData.count())


### Test cases

In [36]:
visible_results_rf=pickle.load(open('RandomForestResultsVisible.pkl','rb'))
assert Classify_RF(trainingData, testData, 3) <= visible_results_rf['RF_10p_3'] 

In [37]:
assert Classify_RF(trainingData, testData, 6) <= visible_results_rf['RF_10p_6']

In [38]:
#Hidden Tests here
###
### AUTOGRADER TEST - DO NOT REMOVE
###


In [39]:
#Hidden Tests here
###
### AUTOGRADER TEST - DO NOT REMOVE
###


In [40]:
end_nb = time.time()
print("Total time taken: ", end_nb - start_nb)

Total time taken:  3084.7680888175964
